In [ ]:
#https://www.kaggle.com/c/cifar-10


pip install kaggle-cli
#get data from kaggle
kg download -u <username> -p <password> -c cifar-10

p7zip -d train.7z  
p7zip -d test.7z  


In [ ]:
from datetime import datetime
HOMEPATH = "/home/ubuntu/fastai/"
DATA_HOME_DIR = HOMEPATH + "data/cifar-10/"
sampleRatio = 0.2
trainRatio = 0.8
import os

In [ ]:
from keras import backend as K
import os

print("----------------")
def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

In [ ]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *

In [ ]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import pandas as pd
import scipy
from random import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [ ]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [ ]:
#path = HOMEPATH + "data/dogscats/sample/"
path = DATA_HOME_DIR 
#path = DATA_HOME_DIR + "sample/"
train_path = path + 'train/'
test_path = path + 'test/'
testUnknown_path = path + 'test/unknown/'
valid_path = path + 'valid/'
model_path = path + 'models/'
sample_path = path + 'sample/'
sampleTrain_path = path + 'sample/train/'
sampleTest_path = path + 'sample/test/'
sampleValid_path = path + 'sample/valid/'
sampleResults_path = path + 'sample/results/'
sampleTestUnknown_path = path + 'sample/test/unknown/'

categories = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def makeNewDir(newDirPath):
    if not os.path.exists(newDirPath): 
        print("directory ", newDirPath, " did not exist, creating:")
        os.mkdir(newDirPath)
    else:
        print("directory ", newDirPath, "already existed.")

#makeNewDir(train_path)#already exists after unzipping
#makeNewDir(test_path)#already exists after unzipping

makeNewDir(testUnknown_path)
makeNewDir(valid_path)
makeNewDir(model_path)
makeNewDir(sample_path)
makeNewDir(sampleTrain_path)
makeNewDir(sampleTest_path)
makeNewDir(sampleValid_path)
makeNewDir(sampleResults_path)
makeNewDir(sampleTestUnknown_path)
for category in categories:
    newDir = train_path + category
    makeNewDir(newDir)
    newDir = valid_path + category
    makeNewDir(newDir)
    newDir = sampleTrain_path + category
    makeNewDir(newDir)
    newDir = sampleValid_path + category
    makeNewDir(newDir)



In [ ]:
#load trainLabels.csv and use to move files to category directories
trainLabelsFile = DATA_HOME_DIR + "trainLabels.csv"
trainLabels = pd.read_csv(trainLabelsFile).as_matrix()
print (type(trainLabels), trainLabels.shape)
print (trainLabels[0:10,])

In [ ]:
#
def moveFilesToCategories(path_):
    os.chdir(path_)
    errCount = 0
    success = 0
    print ("current dir:", os.getcwd())
    for i in range(0,trainLabels.shape[0]):
        #move file to subdirectory 
        #print (trainLabels[i,0], trainLabels[i,1])
        #print("moving:", str(trainLabels[i,0])+".png", " to ", trainLabels[i,1]+"/"+str(trainLabels[i,0])+".png")
        try:
            os.rename(str(trainLabels[i,0])+".png", trainLabels[i,1]+"/"+str(trainLabels[i,0])+".png")
            success += 1
        except:
            errCount += 1
            #print ("Unexpected error:", sys.exc_info()[0])
        #if i>5: break
    print ("success:", success)
    print ("errCount:", errCount)

In [ ]:
moveFilesToCategories(train_path)

In [ ]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]
    

In [ ]:
def moveListToDir(fileList, currDir, destDir):
    print ("moving ", len(fileList), " files from ", currDir, " to ", destDir)
    for file in fileList:
        #move file to destDir
        errCount = 0
        success = 0
        try:
            os.rename(currDir+"/"+file, destDir+"/"+file)
            print (currDir+"/"+file, " to ", destDir+"/"+file)
            success += 1
        except:
            errCount += 1
            print ("error moving ", currDir+"/"+file, " to ", destDir+"/"+file)
            print ("Unexpected error:", sys.exc_info()[0])
        #break
    print ("success:", success)
    print ("errCount:", errCount)


In [ ]:
print (train_path, " has ", len(dirFileList(train_path)), " files.")
#identify files in each category.
#apply split for train | sample, sample/train, sample/valid and valid
for category in categories:
    fileList = dirFileList(train_path+category)
    print (train_path+category, " has ", len(fileList), " files.")
    #randomly select fraction (sampleRatio) of files to move to sample
    shuffle(fileList)
    numSamples = int(len(fileList)*sampleRatio)
    sampleList = fileList[0:numSamples]
    print ("sampleList:", len(sampleList))
    nonValidList =  fileList[numSamples:]
    print ("nonValidList:", len(nonValidList))

    numTrain = int(len(nonValidList)*trainRatio)
    trainList = nonValidList[0:numTrain]#these don't move 
    validList = nonValidList[numTrain:]
    print ("trainList:", len(trainList))
    print ("validList:", len(validList))
    #fix required here. all of sample is moved to sample.
    #all of validList is moved to valid.
    # need to split valid and train after moving sample out.
    moveListToDir(validList, train_path+category, valid_path+category)
    
    #  of this selection, move randomly selected fraction (trainValidRatio)   to sample/valid 
    numSamplesTrain = int(len(sampleList)*trainRatio)
    sampleTrainList = sampleList[0:numSamplesTrain]
    sampleValidList = sampleList[numSamplesTrain:]
    print ("sampleTrainList:", len(sampleTrainList))
    moveListToDir(sampleTrainList, train_path+category, sampleTrain_path+category)
    print ("sampleValidList:", len(sampleValidList))
    moveListToDir(sampleValidList, train_path+category, sampleValid_path+category)


In [ ]:
dirList = [train_path, valid_path, sampleTrain_path, sampleValid_path]

for dir_path in dirList:
    for category in categories:
        print ("# of files in", dir_path+category, ":", len(dirFileList(dir_path+category)))
